In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from til_23_asr import VoiceExtractor
from IPython.display import Audio
import librosa
import torch
from pathlib import Path

In [3]:
# load original and noisy speech files
files = Path('../data/audio/val').glob('*.wav')
wavs = {}
for file in sorted(files):
    noisy, sr = librosa.load(file, sr=None)
    wavs[file.stem] = (noisy, sr)

In [4]:
extractor = VoiceExtractor(
    sr=sr,
    # spectral_gate_std_thres=1.0,
    # spectral_noise_remove=1.0,
    # spectral_n_fft=512,
    # spectral_freq_mask_hz=None,
    # spectral_time_mask_ms=None,
    # skip_spectral=False,
    # skip_demucs=False,
    # use_ori=False,
    return_noise=True,
).to("cuda")

In [5]:
for name, (noisy, sr) in wavs.items():
    print(name)
    display(Audio(noisy, rate=sr))
    clean, new_sr, noise = extractor(torch.tensor(noisy, device="cuda"), sr)
    clean = clean.numpy(force=True)
    display(Audio(clean, rate=new_sr))
    display(Audio(noise.numpy(force=True), rate=new_sr))
    

In [ ]:
demucked, d_sr = extractor._demucs(torch.tensor(noisy), sr)
display(Audio(demucked, rate=d_sr))

In [ ]:
display(Audio(extractor._spectral(demucked, d_sr)[0], rate=sr))